In [23]:
import rpy2.robjects as ro
import pandas as pd
import json

r = ro.r
r['source']('ParseNFLPlayers.R')

parse_nfl_player_r = ro.globalenv['get_player_data']
parse_position_players_r = ro.globalenv['get_position_players']

In [24]:
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# This is an R "tibble" of the data
# We load this object from the R script we used earlier
player_stats_r = ro.globalenv['player_stats']
team_data_r = ro.globalenv['team_data']
player_stats_kicker_r = ro.globalenv['player_stats_kicker']
roster_r = ro.globalenv['roster']
official_player_stats_r = ro.globalenv['official_player_stats_total']
weekly_stats = ro.globalenv['official_player_stats']

# Convert the tibble to a pandas data frame
with localconverter(ro.default_converter + pandas2ri.converter):
    df_player_stats = ro.conversion.rpy2py(player_stats_r)
    df_team_data = ro.conversion.rpy2py(player_stats_r)
    df_player_stats_kicker = ro.conversion.rpy2py(player_stats_kicker_r)
    df_roster = ro.conversion.rpy2py(roster_r)
    df_official_player_stats = ro.conversion.rpy2py(official_player_stats_r)
    df_weekly_stats = ro.conversion.rpy2py(weekly_stats)


In [17]:
team_passing_tds = {}
for index, recent_team in df_official_player_stats.iterrows():
    if(df_official_player_stats["recent_team"][index] not in team_passing_tds.keys()):
        team_passing_tds[df_official_player_stats["recent_team"][index]] = 0
    team_passing_tds[df_official_player_stats["recent_team"][index]] += df_official_player_stats["passing_tds"][index]
print(team_passing_tds)

team_passing_yards = {}
for index, recent_team in df_official_player_stats.iterrows():
    if(df_official_player_stats["recent_team"][index] not in team_passing_yards.keys()):
        team_passing_yards[df_official_player_stats["recent_team"][index]] = 0
    team_passing_yards[df_official_player_stats["recent_team"][index]] += df_official_player_stats["passing_yards"][index]
print(team_passing_yards)

{'TB': 46, 'ARI': 27, 'PIT': 25, 'GB': 39, 'WAS': 21, 'BAL': 24, 'SEA': 30, 'HOU': 21, 'ATL': 20, 'NYJ': 17, 'LV': 24, 'KC': 49, 'LA': 50, 'LAC': 38, 'NE': 26, 'BUF': 45, 'CHI': 16, 'SF': 28, 'CAR': 14, 'TEN': 24, 'NO': 29, 'NYG': 15, 'MIN': 34, 'CLE': 21, 'JAX': 12, 'IND': 27, 'DAL': 41, 'DET': 23, 'PHI': 21, 'MIA': 21, 'DEN': 20, 'CIN': 42}
{'TB': 5983.0, 'ARI': 4756.0, 'PIT': 4232.0, 'GB': 4751.0, 'WAS': 3746.0, 'BAL': 4601.0, 'SEA': 3815.0, 'HOU': 3630.0, 'ATL': 3987.0, 'NYJ': 3625.0, 'LV': 5118.0, 'KC': 5996.0, 'LA': 6121.0, 'LAC': 5014.0, 'NE': 4330.0, 'BUF': 5087.0, 'CHI': 3635.0, 'SF': 4972.0, 'CAR': 3573.0, 'TEN': 3965.0, 'NO': 3437.0, 'NYG': 3463.0, 'MIN': 4450.0, 'CLE': 3619.0, 'JAX': 3674.0, 'IND': 3588.0, 'DAL': 5233.0, 'DET': 3884.0, 'PHI': 3843.0, 'MIA': 3936.0, 'DEN': 3856.0, 'CIN': 5917.0}


In [25]:
df_official_player_stats["percent_rec_tds"] = 0
for index, receving_tds in df_official_player_stats.iterrows():
    team = str(df_official_player_stats["recent_team"][index])
    df_official_player_stats["percent_rec_tds"][index] = df_official_player_stats["receiving_tds"][index] / team_passing_tds[team]

df_official_player_stats["percent_rec_yards"] = 0
for index, receving_yards in df_official_player_stats.iterrows():
    team = str(df_official_player_stats["recent_team"][index])
    df_official_player_stats["percent_rec_yards"][index] = df_official_player_stats["receiving_yards"][index] / team_passing_yards[team]

df_official_player_stats.sort_values(by='receiving_yards', ascending=False)

/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_23882/558310768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_official_player_stats["percent_rec_tds"][index] = df_official_player_stats["receiving_tds"][index] / team_passing_tds[team]
/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_23882/558310768.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_official_player_stats["percent_rec_yards"][index] = df_official_player_stats["receiving_yards"][index] / team_passing_yards[team]


,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,percent_rec_tds,percent_rec_yards
274,00-0033908,C.Kupp,21,LA,0,2,0.0,0,0.0,0.0,...,1,1.202281,0.322535,0.328025,0.713420,0.0,376.80,554.80,0.440000,0.396177
630,00-0036900,J.Chase,21,CIN,0,0,0.0,0,0.0,0.0,...,0,0.933914,0.237546,0.367239,0.613386,0.0,269.60,375.60,0.333333,0.308095
100,00-0031381,D.Adams,17,GB,0,0,0.0,0,0.0,0.0,...,0,0.948614,0.322131,0.384473,0.752328,0.0,230.30,362.30,0.282051,0.345822
545,00-0036322,J.Jefferson,17,MIN,2,4,35.0,0,0.0,0.0,...,0,0.780299,0.294111,0.458190,0.761899,0.0,222.40,330.40,0.294118,0.363146
475,00-0035719,D.Samuel,19,SF,1,2,24.0,1,0.0,0.0,...,0,1.469369,0.251392,0.254537,0.555265,0.0,303.06,390.06,0.250000,0.313556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,00-0035428,D.Helm,2,LV,0,0,0.0,0,0.0,0.0,...,0,-1.000000,0.027047,0.000775,0.041113,0.0,-0.10,0.90,0.000000,-0.000195
125,00-0031800,T.Heinicke,16,WAS,321,494,3419.0,20,15.0,38.0,...,0,0.000000,0.041667,-0.024242,0.045530,0.0,221.86,222.86,0.000000,-0.000534
43,00-0029164,M.Remmers,1,KC,0,0,0.0,0,0.0,0.0,...,0,0.000000,0.029412,0.000000,0.044118,0.0,-0.20,0.80,0.000000,-0.000334
94,00-0031319,D.Spencer,4,DEN,0,0,0.0,0,0.0,0.0,...,0,-0.333333,0.034316,0.011005,0.059178,0.0,-0.30,0.70,0.000000,-0.000778


In [26]:
df_official_player_stats["rec_dom"] = 0
for index, receving_yards in df_official_player_stats.iterrows():
    df_official_player_stats["rec_dom"][index] = (1/3)*(df_official_player_stats["percent_rec_yards"][index] + df_official_player_stats["percent_rec_tds"][index] + df_official_player_stats["target_share"][index])

df_official_player_stats["rec_share"] = 0
df_official_player_stats["rec_share_%"] = 0
for index, receving_yards in df_official_player_stats.iterrows():
    df_official_player_stats["rec_share"][index] = (1/4)*(df_official_player_stats["percent_rec_yards"][index]) + (1/2)*(df_official_player_stats["percent_rec_tds"][index]) + (1/4)*(df_official_player_stats["target_share"][index])
    df_official_player_stats["rec_share_%"][index] = "{:.0%}".format(df_official_player_stats["rec_share"][index])


df_official_player_stats.sort_values(by='rec_share', ascending=False)

/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_23882/793526572.py:3: RuntimeWarning: invalid value encountered in double_scalars
  df_official_player_stats["rec_dom"][index] = (1/3)*(df_official_player_stats["percent_rec_yards"][index] + df_official_player_stats["percent_rec_tds"][index] + df_official_player_stats["target_share"][index])
/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_23882/793526572.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_official_player_stats["rec_dom"][index] = (1/3)*(df_official_player_stats["percent_rec_yards"][index] + df_official_player_stats["percent_rec_tds"][index] + df_official_player_stats["target_share"][index])
/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_23882/793526572.py:8: RuntimeWarning: invalid value encountered in

,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,percent_rec_tds,percent_rec_yards,rec_dom,rec_share,rec_share_%
274,00-0033908,C.Kupp,21,LA,0,2,0.0,0,0.0,0.0,...,0.328025,0.713420,0.0,376.80,554.80,0.440000,0.396177,0.386237,0.399678,40%
460,00-0035640,D.Metcalf,17,SEA,0,0,0.0,0,0.0,0.0,...,0.386269,0.686479,0.0,169.30,244.30,0.400000,0.253473,0.310289,0.332717,33%
350,00-0034753,M.Andrews,17,BAL,0,0,0.0,0,0.0,0.0,...,0.323770,0.613314,0.0,194.10,301.10,0.375000,0.295805,0.309530,0.325897,33%
402,00-0035216,D.Johnson,17,PIT,0,0,0.0,0,0.0,0.0,...,0.334945,0.648333,0.0,176.80,288.80,0.360000,0.282372,0.306096,0.319572,32%
545,00-0036322,J.Jefferson,17,MIN,2,4,35.0,0,0.0,0.0,...,0.458190,0.761899,0.0,222.40,330.40,0.294118,0.363146,0.317125,0.311373,31%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,00-0036971,T.Lawrence,17,JAX,359,602,3641.0,12,17.0,32.0,...,NaN,NaN,0.0,199.04,199.04,0.000000,0.000000,NaN,NaN,nan%
647,00-0036972,M.Jones,18,NE,376,559,4033.0,24,15.0,31.0,...,NaN,NaN,0.0,240.02,240.02,0.000000,0.000000,NaN,NaN,nan%
649,00-0036982,C.Golston,1,DAL,0,0,0.0,0,0.0,0.0,...,NaN,NaN,1.0,6.00,6.00,0.000000,0.000000,NaN,NaN,nan%
651,00-0036986,N.Wright,1,DAL,0,0,0.0,0,0.0,0.0,...,NaN,NaN,1.0,6.00,6.00,0.000000,0.000000,NaN,NaN,nan%


In [30]:
df_sorted_rec_share = df_official_player_stats.sort_values(by='rec_share', ascending=False)

df_sorted_rec_share.head(50)

,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,percent_rec_tds,percent_rec_yards,rec_dom,rec_share,rec_share_%
274,00-0033908,C.Kupp,21,LA,0,2,0.0,0,0.0,0.0,...,0.328025,0.713420,0.0,376.80,554.80,0.440000,0.396177,0.386237,0.399678,40%
460,00-0035640,D.Metcalf,17,SEA,0,0,0.0,0,0.0,0.0,...,0.386269,0.686479,0.0,169.30,244.30,0.400000,0.253473,0.310289,0.332717,33%
350,00-0034753,M.Andrews,17,BAL,0,0,0.0,0,0.0,0.0,...,0.323770,0.613314,0.0,194.10,301.10,0.375000,0.295805,0.309530,0.325897,33%
402,00-0035216,D.Johnson,17,PIT,0,0,0.0,0,0.0,0.0,...,0.334945,0.648333,0.0,176.80,288.80,0.360000,0.282372,0.306096,0.319572,32%
545,00-0036322,J.Jefferson,17,MIN,2,4,35.0,0,0.0,0.0,...,0.458190,0.761899,0.0,222.40,330.40,0.294118,0.363146,0.317125,0.311373,31%
100,00-0031381,D.Adams,17,GB,0,0,0.0,0,0.0,0.0,...,0.384473,0.752328,0.0,230.30,362.30,0.282051,0.345822,0.316668,0.308014,31%
630,00-0036900,J.Chase,21,CIN,0,0,0.0,0,0.0,0.0,...,0.367239,0.613386,0.0,269.60,375.60,0.333333,0.308095,0.292991,0.303077,30%
363,00-0034827,Dj.Moore,17,CAR,0,0,0.0,0,0.0,0.0,...,0.418224,0.719705,0.0,144.50,237.50,0.285714,0.323818,0.298055,0.294970,29%
383,00-0034983,H.Renfrow,18,LV,0,0,0.0,0,0.0,0.0,...,0.172012,0.437311,0.0,161.90,272.90,0.375000,0.214146,0.266805,0.293854,29%
88,00-0031236,B.Cooks,16,HOU,0,0,0.0,0,0.0,0.0,...,0.394166,0.684067,0.0,141.80,231.80,0.285714,0.285675,0.281163,0.282301,28%
